# Modelo de inteligência artificial para reconhecimento de escoamento

## 1. Carregar os dados

In [10]:
import numpy as np 
import cv2
import os

In [11]:
PastaPrincipal = '../Dados'
PastasSecundarias = os.listdir(PastaPrincipal)
num__classes = len(PastasSecundarias)
print(f'{num__classes} Labels criadas: {PastasSecundarias}')


2 Labels criadas: ['com_chapeu', 'sem_chapeu']


## 2. Atribuindo cada labels para cada imagem

In [14]:
imagens = []
labels = []
print('Importando as labels ...')
for label in PastasSecundarias:
    lista__de_imagens = os.listdir(PastaPrincipal + '/' + str(label))
    for imagem in lista__de_imagens:
        imagens_atual = cv2.imread(PastaPrincipal + '/' + str(label) + '/' + imagem)
        imagens_atual = cv2.resize(imagens_atual,(32,32))
        imagens.append(imagens_atual)
        labels.append(label)
    print(label, end=" ")



Importando as labels ...
com_chapeu sem_chapeu 

In [ ]:
print('Quantidade de imagens carregadas: ',len(imagens))